In [1]:
import pandas as pd

def load_some_data(path, separ = ','):
    return pd.read_csv(path, sep=separ, parse_dates = ['datetime'], index_col = 'datetime')

events_battle = load_some_data(r'data/events_battle.txt')
events_hero = load_some_data(r'data/events_hero.txt')
events_level = load_some_data(r'data/events_level.txt', '\t')
events_payment = pd.read_csv(r'data/events_payment.txt', parse_dates = ['created'], index_col = 'created')
events_quest = load_some_data(r'data/events_quest.txt')
events_resource = load_some_data(r'data/events_resource.txt')
events_user = load_some_data(r'data/events_user.txt')

In [2]:
def get_user_res():
    used_cols = ['user_id']
    all_dates = events_battle[used_cols].copy()
    all_dates = all_dates.append(events_resource[used_cols])
    all_dates = all_dates.append(events_quest[used_cols])
    all_dates = all_dates.append(events_payment[used_cols])
    last_dates = all_dates.reset_index().set_index(['user_id']).groupby(['user_id']).agg('max')
    last_dates.rename(index=str, columns={"index": "datetime"}, inplace = True)
    
    def is_leaved(date):
        current_date = pd.Timestamp(2018, 11, 8, 12)
        if (current_date - date).days > 3:
            return 1
        else:
            return 0
    last_dates['leaved'] = last_dates.apply(lambda dat : is_leaved(dat['datetime']),axis = 1)
    last_dates.rename(index=str, columns={"index": "datetime"}, inplace = True)
    return last_dates

In [3]:
def get_events_data():
    battle_subset = events_battle.copy()
    grouped_battle_subset = battle_subset.groupby([battle_subset.index.date, 'user_id']).agg('count')
    grouped_battle_subset = grouped_battle_subset[['id']]
    grouped_events = []
    events = [
    events_battle,
    events_quest
    ]
    for e in events:
        e = e.groupby([e.index.date, 'user_id']).agg('count')
        e = e[['id']]
        grouped_events.append(e)
    counted_events = grouped_events[0]
    counted_events['quests_count'] = grouped_events[1]['id']
    counted_events.rename(columns={"id": "battles_count"}, inplace=True)
    counted_events.fillna(0, inplace=True)
    counted_events['quests_count'] = counted_events['quests_count'].astype('int64')
    return counted_events


In [4]:
def get_battle_data():
    first = events_battle[['user_id',"hero_id", 'status', 'opponent_type', 'bot_template_id', 'opponent_race_id', 'kind']].copy()
    second = first.copy()
    res = pd.merge(first, second, how='inner', left_index=True, right_index=True)
    res = res.loc[res["user_id_x"] != res["user_id_y"]]
    res = res.loc[res["status_x"]!=res["status_y"]]
    return res

In [5]:
leaved = get_user_res()[['leaved']]
events = get_events_data()
battles = get_battle_data()


In [6]:
kek = battles[['user_id_x', 'status_x', 'opponent_type_x', 'kind_x']].copy()
for col in kek.columns:
    if (col == 'user_id_x'):
        continue
    kek[col] = pd.Categorical(kek[col]).codes
res = pd.DataFrame()
res[['winrate']] = kek.groupby('user_id_x')[['status_x']].mean() / 2
res[['oponents']] =  kek.groupby('user_id_x')[['opponent_type_x']].mean()
res[['battle_types']] = kek.groupby('user_id_x')[['kind_x']].mean()
res[['total_count']] = kek.groupby('user_id_x')[['kind_x']].count()

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
clf = RandomForestClassifier(n_estimators=100, max_depth=3,  random_state=0)
res.index.intersection(leaved.index)
leaved = leaved.loc[res.index.intersection(leaved.index)]
clf = clf.fit(res, leaved['leaved'])

In [8]:
clf.estimators_[1]

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False,
            random_state=398764591, splitter='best')

In [9]:
from sklearn.tree import export_graphviz
from IPython import display
from sklearn.ensemble import RandomForestRegressor


clf = clf.fit(res, leaved['leaved'])
str_tree = export_graphviz(clf.estimators_[1], 
   out_file=None, 
   feature_names=res.columns, # column names
   filled=True,        
   special_characters=True, 
   rotate=True)



In [10]:
from sklearn.tree import export_graphviz
export_graphviz(clf.estimators_[1], out_file='tree_limited.dot', feature_names = res.columns,
                rounded = True, proportion = False, precision = 2, filled = True)


In [1]:
from graphviz import Source 
from IPython.display import SVG 
from sklearn import tree 
# extra step to allow graphviz to be found 
import os
sergey_notebook_path = r'D:\anaconda\envs\foo\Library\bin\graphviz'
sergey_desktop_path = r'F:\Anaconda\envs\foo\Library\bin\graphviz'
os.environ["PATH"] += os.pathsep + sergey_desktop_path
graph = Source( tree.export_graphviz(clf.estimators_[5], out_file=None, feature_names=res.columns)) 
SVG(graph.pipe(format='svg'))

NameError: name 'clf' is not defined

In [16]:
res

,winrate,oponents,battle_types,total_count
user_id_x,,,,
001a3472-44bc-4583-95d9-bf8759085017,0.000000,1.000000,1.000000,2
002bcd7d-83b1-42ac-b257-ed71a4585f5e,0.500000,1.000000,1.000000,2
00467ba2-b71c-4a69-b8bc-d7491cf2eca0,0.000000,0.000000,0.000000,1
0047e219-202a-4eda-8d31-b35a58f50eec,0.000000,0.500000,0.000000,2
00695cdd-d981-4e2c-ad82-f86059a0013d,1.000000,0.000000,0.000000,1
00a21109-ed6d-4be2-8e20-b79158da28b8,0.333333,0.666667,0.666667,3
00c7d924-83f5-4679-9d96-35356af1f53d,0.584906,0.811321,0.471698,53
00dd3007-71cf-42de-9c46-680988d89242,0.000000,1.000000,0.500000,2
010a6e53-245a-47a3-8e44-c8212c101399,0.000000,1.000000,1.000000,2
